<a href="https://colab.research.google.com/github/drfperez/openair/blob/main/SIVIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import time
import urllib.parse
from google.colab import files
from datetime import datetime

def descarregar_filtrat(any_seleccionat=None, codi_regio=None, codi_abs=None):
    """
    Descarrega dades filtrant per any, codi_regio i/o codi_abs.
    """
    base_url = "https://analisi.transparenciacatalunya.cat/resource/fa7i-d8gc.json"
    limit = 50000
    offset = 0
    tots_els_dades = []

    # Construir condicions SoQL
    condicions = []
    if codi_regio:
        condicions.append(f"codi_regio='{codi_regio}'")
    if codi_abs:
        condicions.append(f"codi_abs='{codi_abs}'")
    if any_seleccionat:
        data_inici = f"{any_seleccionat}-01-01T00:00:00"
        data_fi = f"{any_seleccionat+1}-01-01T00:00:00"
        condicions.append(f"data >= '{data_inici}' AND data < '{data_fi}'")

    where_clause = " AND ".join(condicions) if condicions else "1=1"  # 1=1 per agafar tot si no hi ha filtre

    print("🚀 Iniciant descàrrega amb els filtres:")
    print(f"   Any: {any_seleccionat}, codi_regio: {codi_regio}, codi_abs: {codi_abs}")

    while True:
        query = f"?$where={where_clause}&$limit={limit}&$offset={offset}&$order=data ASC"
        url_neta = base_url + urllib.parse.quote(query, safe='?=&-:T')

        try:
            df_bloc = pd.read_json(url_neta)

            if df_bloc.empty:
                break

            tots_els_dades.append(df_bloc)
            offset += limit

            ultima_data = df_bloc['data'].iloc[-1]
            print(f"📥 {offset} registres obtinguts... Arribant a la data: {ultima_data[:10]}")

            time.sleep(0.5)

        except Exception as e:
            print(f"⚠️ Error durant la descàrrega: {e}")
            break

    if tots_els_dades:
        df_final = pd.concat(tots_els_dades, ignore_index=True)
        df_final['data'] = pd.to_datetime(df_final['data'])

        print(f"\n✅ ÈXIT: {len(df_final)} registres descarregats.")
        print(f"📅 Període: {df_final['data'].min().date()} fins a {df_final['data'].max().date()}")

        # Nom de fitxer basat en filtres
        nom_fitxer = f"SIVIC"
        if codi_regio: nom_fitxer += f"_regio_{codi_regio}"
        if codi_abs: nom_fitxer += f"_abs_{codi_abs}"
        if any_seleccionat: nom_fitxer += f"_{any_seleccionat}"
        nom_fitxer += ".csv"

        df_final.to_csv(nom_fitxer, index=False)
        files.download(nom_fitxer)

        return df_final
    else:
        print("❌ No s'han pogut recuperar dades.")
        return None

# Exemple d'ús
any_usuari = input("Introdueix l'any (deixa buit per tots): ").strip()
codi_regio_usuari = input("Introdueix codi_regio (deixa buit per tots): ").strip()
codi_abs_usuari = input("Introdueix codi_abs (deixa buit per tots): ").strip()

any_usuari = int(any_usuari) if any_usuari else None
codi_regio_usuari = codi_regio_usuari if codi_regio_usuari else None
codi_abs_usuari = codi_abs_usuari if codi_abs_usuari else None

df_resultat = descarregar_filtrat(any_usuari, codi_regio_usuari, codi_abs_usuari)